In [8]:
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Reshape, Dropout, Conv2D, Activation
from keras import backend as K
from keras.optimizers import Adam

In [9]:
alpha = 1.0
n_classes = 2
dropout = 1e-3

# Load MobileNet trained to convergence on ImageNet without the final fully connected layer.

base_model = MobileNet(input_shape=(96, 96, 3),
                       alpha=alpha,
                       depth_multiplier=1,
                       dropout=dropout,
                       include_top=False,
                       weights=None, #'imagenet',
                       pooling=None)

# Add the final layers.

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Reshape((1, 1, int(1024 * alpha)), name='reshape_1')(x)
x = Dropout(dropout, name='dropout')(x)
x = Conv2D(n_classes,
           (1, 1),
           padding='same',
           name='conv_preds')(x)
x = Activation('softmax', name='act_softmax')(x)
x = Reshape((n_classes,), name='reshape_2')(x)

model = Model(inputs=base_model.input, outputs=x)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy')